In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

In [2]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities_v2 import *

In [3]:
%matplotlib widget

In [4]:
%reload_ext autoreload
%autoreload 2

### visual inspection

In [ ]:
## force template
dist = 11.4
z_sep = -15.85
y_offset = 4.9
test = force_vs_time(dist,z_sep,200,3,"x",20, yuk_or_grav="yuk",offset_y=y_offset, 
                     alpha=1e8)
def fft_norm(N, fsamp):
    "Factor to normalize FFT to ASD units"
    return np.sqrt(2 / (N * fsamp))
scale = np.sqrt(2)*fft_norm(5000, 5000)

_,ax = plt.subplots()
x = list(test[1])
fft2 = np.abs(np.fft.rfft(x))*scale
freq1 = np.fft.rfftfreq(len(x), d=1./5000)
ax.semilogy(freq1, fft1, label='7.6 $\mu m$ sphere - Z=-15.7')
ax.set(xlim=(1,100), ylim=(None,None), xlabel='frequency [Hz]', ylabel='Force [N]', title=r'$\alpha=1x10^{8}$')
ax.legend()

In [9]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/New_Discharge/',
                         start_file=0, max_file=1000)

No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
15  files in folder
15  files loaded


In [10]:
bdf_z = bdf_discharge[:5]
bdf_x = bdf_discharge[5:10]
bdf_y = bdf_discharge[10:]

freq = bdf_z[0].psd2('z')[0]
z_scale = bdf_z[0].psd2('z')[1][freq==71]
for bb in bdf_z:
    z_scale += bb.psd2('z')[1][freq==71]
z_scale /= len(bdf_z)

x_scale = bdf_x[0].psd2('x')[1][freq==71]
for bb in bdf_x:
    x_scale += bb.psd2('x')[1][freq==71]
x_scale /= len(bdf_x)

y_scale = bdf_y[0].psd2('y')[1][freq==71]
for bb in bdf_y:
    y_scale += bb.psd2('y')[1][freq==71]
y_scale /= len(bdf_y)

charges = 7
force = charges * 1.6e-19 * 20 / 8.6e-3 * 0.63 # in Newtons
z_scale = force/np.sqrt(z_scale)
x_scale = force/np.sqrt(x_scale)
y_scale = force/np.sqrt(y_scale)

In [37]:
bdf_z = bdf_discharge[:5]

freq = bdf_z[0].psd2('z')[0]
z_scale2 = np.sqrt(bdf_z[0].psd2('z')[1][freq==71])
for bb in bdf_z:
    z_scale2 += np.sqrt(bb.psd2('z')[1][freq==71])
z_scale2 /= len(bdf_z)

charges = 7
force = charges * 1.6e-19 * 20 / 8.6e-3 * 0.63 # in Newtons
z_scale2 = force/z_scale2

In [ ]:
r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/NoShaking_1

In [27]:
## 100 files of shaking data for nose floor
# data_bdfs = load_dir(dirname='/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/',
#                          file_prefix='NoShaking_1', start_file=0, max_file=1000)

# 100 files of shaking data for nose floor
data_bdfs = load_dir(dirname='/data/new_trap/20200320/Bead1/Shaking/Shaking378/',
                         file_prefix='Shaking3_', start_file=5000, max_file=1000)
freq = data_bdfs[0].psd2('x')[0]
fftx9 = data_bdfs[0].psd2('x')[1]
ffty9 = data_bdfs[0].psd2('y')[1]
fftz9 = data_bdfs[0].psd2('z')[1]
for i in range(1,1000):
    fftx9 += data_bdfs[i].psd2('x')[1]
    fftz9 += data_bdfs[i].psd2('z')[1]
    ffty9 += data_bdfs[i].psd2('y')[1]
fftx9/=1000
ffty9/=1000
fftz9/=1000

10000  files in folder
1000  files loaded


In [36]:
def func_fft(i):
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    fname = r'/data/new_trap/20200320/Bead1/Discharge/New_Discharge/Discharge_'+str(i)'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    x = bb.z2
    fft_ = np.fft.rfft(x,norm='ortho')
#     freq_ = np.fft.rfftfreq(len(x), d=1./5000)
    return (fft_)

SyntaxError: invalid syntax (<ipython-input-36-be6f3fc24591>, line 3)

In [56]:
test_norms = np.array(Parallel(n_jobs=20)(delayed(func_fft)(i) for i in tqdm(range(10000))))

100%|██████████| 10000/10000 [02:09<00:00, 68.13it/s]


In [60]:
freq_ = np.fft.rfftfreq(len(x), d=1./5000)

In [66]:
np.angle(np.mean(test_norms, axis=0)[freq_==33])

array([-0.98156684])

In [58]:
_,ax = plt.subplots()
ax.semilogy(freq_, np.abs(np.mean(test_norms, axis=0))*z_scale2)
_ = ax.set(xlim=(5,40), xlabel='Frequency [Hz.]', title=r'Z force')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [261]:
del data_bdfs

In [21]:
_,ax = plt.subplots(figsize=(9,4))
# ax.semilogy(freq, np.sqrt(fftz4)*z_scale, label='Wilson 378-3')
ax.semilogy(freq, np.sqrt(fftx6)*x_scale, label='Wilson 378-3')
ax.semilogy(freq1, np.sqrt(fftx5)*x_scale+fft2/np.sqrt(10), '--', label=r'noise+signal[$\alpha=10^8$]', alpha=0.8)
ax.semilogy(freq1, np.sqrt(fftx5)*x_scale, 'r--', label='noise', alpha=0.5)
ax.legend()
_ = ax.set(xlim=(5,40), ylim=(1e-17,7e-15), xlabel='Frequency [Hz.]', title=r'Z force')
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
_,ax = plt.subplots(figsize=(9,4))
# ax.semilogy(freq, np.sqrt(fftz4)*z_scale, label='Wilson 378-3')
# ax.semilogy(freq, np.sqrt(fftz6)*z_scale, label='Wilson 378-3')
# ax.semilogy(freq, np.sqrt(fftz7)*z_scale, label='Wilson 378-3')
# ax.semilogy(freq, np.sqrt(fftz8)*z_scale, label='Wilson 378-3')
ax.semilogy(freq, np.sqrt((fftz9+fftz8+fftz7+fftz6)/4)*z_scale, label='Wilson 378-3')
ax.semilogy(freq1, np.sqrt(fftz5)*z_scale+fft1/np.sqrt(10), '--', label=r'noise+signal[$\alpha=10^8$]', alpha=0.8)
ax.semilogy(freq1, np.sqrt(fftz5)*z_scale, 'r--', label='noise', alpha=0.5)
ax.legend()
_ = ax.set(xlim=(5,40), ylim=(1e-17,2e-16), xlabel='Frequency [Hz.]', title=r'Z force')
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Profile Likelihood

In [264]:
df = pd.read_pickle("/data/new_trap_processed/processed_files/20200320/Bead1/Shaking/Shaking378/Shaking3_aux.pkl") #here one could put the respective file name
height_cal = df["z_distance_emp"]
time = df['Time_Epoch']
time -= time[0]

In [265]:
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_X_m300k_250s_1hz.h5'
tf_x = BeadDataFile.BeadDataFile(fname)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Y_m300k_250s_1hz.h5'
tf_y = BeadDataFile.BeadDataFile(fname)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
gfw.build_transfer_function([tf_x, tf_y, tf_z], base_freq=1, number_of_harmonics=90, scale_freq=71, plot=False)

No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data


In [266]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/Discharge_after_Mass_20200402/After_spinning_again_20200403/',
                         start_file=230, max_file=40)
bdf_z = bdf_discharge[5:35]

530  files in folder
40  files loaded


In [267]:
bandwidth = 2
m1_zscale = gfw.build_z_response(bdf_z, drive_freq=71, charges=6, bandwidth=bandwidth, decimate=10)

***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  175.2406092149549
***************************************************
Z2-amplitude:  2.61e+02
reduced chi2:  392.6832937186789
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  233.0708308067062
***************************************************
Z2-amplitude:  2.79e+02
reduced chi2:  341.818416019317
***************************************************
Z2-amplitude:  2.81e+02
reduced chi2:  320.1151243984675
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  235.2380260183528
***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  250.1490478570203
***************************************************
Z2-amplitude:  2.66e+02
reduced chi2:  270.7275591716337
***************************************************
Z2-amplitude:  2.67e+02
reduced chi2:  318.6342737410096
********************

In [120]:
## likelihood calculator stuff
gravity_freqs = np.array([9, 15])
# gravity_freqs = np.array([36, 33, 12, 18, 21])
# gravity_freqs = np.array([21, 24, 27, 33])
gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs, gfw.tf_freq, gfw.tf_ffts[2])

In [121]:
noises_std = np.ones(len(gravity_freqs))
noises_std, gravity_freqs

(array([ 1.,  1.]), array([ 9, 15]))

In [642]:
i = 100
fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)

dist = 392.4-np.mean(bb.cant_pos[0])*50-7.6/2
z_sep = np.mean(height_cal)
y_offset = 4.9
stroke = np.std(bb.cant_pos[1])*np.sqrt(2)*50*2
G_lambda = 20

test_shaking378 = force_vs_time(dist,z_sep,stroke,bb.cant_freq,"z",G_lambda, 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
shaking378_template = np.array(test_shaking378[1])

Loading Gravity Data... Done!
51 1.99526231497e-05
Loaded Yukawa Force


In [98]:
ll = likelihood_analyser.LikelihoodAnalyser()
def func1(i):
# for i,bdf_ in enumerate(data_bdfs2):
    print(i,'  ******************')
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/NoShaking_1_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    fit_kwargs = {'A': 0, 'phi': 0.2, 'sigma':13.85,
              'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
              'limit_phi': [0, 2 * np.pi],
              'limit_A': [-1000, 1000], 'limit_sigma': [0, 1000000],
              'print_level': 0, 'fix_A': True, 'fix_phi': True, 'fix_sigma': False}
    

    mm_tmp = ll.find_mle_multiHarmoincs(x=bb.z2, template=shaking378_template, scales=gravity_scales, 
                                          signal_freqs=gravity_freqs, bandwidth=bandwidth, noises=noises_std/noises_std[0],
                                          decimate=10, **fit_kwargs)
    
    return mm_tmp.fval

In [99]:
mm_PL_1 = np.array(Parallel(n_jobs=20)(delayed(func1)(i) for i in tqdm(range(1000))))

100%|██████████| 1000/1000 [00:22<00:00, 41.80it/s]


In [100]:
import scipy.stats as stats

logL = 2*(mm_PL_1-mm_PL_2)*10  ## original
_,ax = plt.subplots()
ax.hist(logL, bins=50, range=(0,6), alpha=1, log=False, label='no noise std')
x = np.arange(0, 6, .05)
plt.plot(x, stats.chi2.pdf(x, df=1)*1000/50*6, color='r', lw=2, label='scaled chi square')
ax.legend()
ax.set(xlabel=r'-2*log($\lambda (\alpha_0 = 0)$)')
print(np.sum(logL>2.71)/1000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.04


### Example

In [601]:
def func_sideband(i):
    print(i,'  ******************')
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    freq_tmp = 36
    return np.std(bb.response_at_freq2('z', drive_freq=freq_tmp+1.5, bandwidth=bandwidth))

In [602]:
sideband_36 = np.array(Parallel(n_jobs=40)(delayed(func_sideband)(i) for i in tqdm(range(10000))))








  0%|          | 0/10000 [00:00<?, ?it/s]






  2%|▏         | 160/10000 [00:00<00:19, 513.01it/s]






  2%|▏         | 240/10000 [00:00<00:18, 527.53it/s]






  3%|▎         | 320/10000 [00:00<00:18, 524.74it/s]






  4%|▍         | 400/10000 [00:00<00:17, 547.81it/s]






  5%|▍         | 480/10000 [00:00<00:15, 599.55it/s]






  6%|▌         | 560/10000 [00:01<00:17, 529.83it/s]






  6%|▋         | 640/10000 [00:01<00:18, 497.92it/s]






  7%|▋         | 720/10000 [00:01<00:18, 511.07it/s]






  8%|▊         | 800/10000 [00:01<00:17, 513.64it/s]






  9%|▉         | 880/10000 [00:01<00:16, 546.11it/s]






 10%|▉         | 960/10000 [00:01<00:17, 527.69it/s]






 10%|█         | 1040/10000 [00:01<00:16, 531.89it/s]






 11%|█         | 1120/10000 [00:02<00:16, 541.64it/s]






 12%|█▏        | 1200/10000 [00:02<00:14, 587.03it/s]






 13%|█▎        | 1280/10000 [00:02<00:13, 623.81it/s]






 14%|█▎        | 1360/10000 [00:02<00:13, 627.47it/s]




In [603]:
sideband_9_mean = [np.mean(sideband_9[i*100:(i+1)*100]) for i in range(100)]
sideband_9_std = [np.std(sideband_9[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_12_mean = [np.mean(sideband_12[i*100:(i+1)*100]) for i in range(100)]
sideband_12_std = [np.std(sideband_12[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_15_mean = [np.mean(sideband_15[i*100:(i+1)*100]) for i in range(100)]
sideband_15_std = [np.std(sideband_15[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_18_mean = [np.mean(sideband_18[i*100:(i+1)*100]) for i in range(100)]
sideband_18_std = [np.std(sideband_18[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_21_mean = [np.mean(sideband_21[i*100:(i+1)*100]) for i in range(100)]
sideband_21_std = [np.std(sideband_21[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_24_mean = [np.mean(sideband_24[i*100:(i+1)*100]) for i in range(100)]
sideband_24_std = [np.std(sideband_24[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_33_mean = [np.mean(sideband_33[i*100:(i+1)*100]) for i in range(100)]
sideband_33_std = [np.std(sideband_33[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

sideband_36_mean = [np.mean(sideband_36[i*100:(i+1)*100]) for i in range(100)]
sideband_36_std = [np.std(sideband_36[i*100:(i+1)*100])/np.sqrt(100) for i in range(100)]

In [119]:
i = 0
fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)

_,ax = plt.subplots()
tmp_freq, tmp_psd = bb.psd2('z')
ax.semilogy(tmp_freq, np.sqrt(tmp_psd)*z_scale)
_ = ax.set(xlim=(5,40), ylim=(1e-17,2e-16), xlabel='Frequency [Hz.]', title=r'Z force')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [565]:
ll = likelihood_analyser.LikelihoodAnalyser()
def func2(i):
    print(i,'  ******************')
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/NoShaking_1_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)

    fit_kwargs = {'A': 0, 'phi': 0.2, 'sigma':13.85,
              'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
              'limit_phi': [0, 2 * np.pi],
              'limit_A': [-1000, 1000], 'limit_sigma': [0, 1000000],
              'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    
    mm_tmp = ll.find_mle_multiHarmoincs(x=bb.z2, template=shaking378_template, scales=gravity_scales, 
                                          signal_freqs=gravity_freqs, bandwidth=bandwidth, noises=noises_std/noises_std[0],
                                          decimate=10, **fit_kwargs)
    
    return mm_tmp.values[0]

In [567]:
alpha_mle_noshaking = np.array(Parallel(n_jobs=40)(delayed(func2)(i) for i in tqdm(range(1000))))







  0%|          | 0/1000 [00:00<?, ?it/s]





  0%|          | 1/1000 [00:06<1:48:19,  6.51s/it]





  8%|▊         | 80/1000 [00:08<1:09:56,  4.56s/it]





 12%|█▏        | 120/1000 [00:09<46:56,  3.20s/it] 





 16%|█▌        | 160/1000 [00:10<31:27,  2.25s/it]





 20%|██        | 200/1000 [00:10<21:02,  1.58s/it]





 24%|██▍       | 240/1000 [00:11<14:04,  1.11s/it]





 28%|██▊       | 280/1000 [00:12<09:24,  1.28it/s]





 32%|███▏      | 320/1000 [00:13<06:16,  1.81it/s]





 36%|███▌      | 360/1000 [00:13<04:12,  2.54it/s]





 40%|████      | 400/1000 [00:14<02:49,  3.54it/s]





 44%|████▍     | 440/1000 [00:15<01:53,  4.93it/s]





 48%|████▊     | 480/1000 [00:16<01:16,  6.75it/s]





 52%|█████▏    | 520/1000 [00:17<00:52,  9.11it/s]





 56%|█████▌    | 560/1000 [00:17<00:36, 12.11it/s]





 60%|██████    | 600/1000 [00:18<00:25, 15.64it/s]





 64%|██████▍   | 640/1000 [00:19<00:18, 19.89it/s]





 68%|██████▊   | 680/1000 [00:20<00:13, 24.38it/s]

In [572]:
np.mean(alpha_mle_noshaking), np.std(alpha_mle_noshaking)

(0.16121808968811946, 8.1622794385656583)

In [729]:
def calc_bdf(i):
# for i in range(0,10):
    print(i,'  ******************')
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/NoShaking_1_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    
    fit_kwargs = {'A': 0, 'phi': 0.2, 'sigma':13.85,
              'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
              'limit_phi': [0, np.pi],
              'limit_A': [-100, 100], 'limit_sigma': [0, 1000000],
              'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    
#     jj = i//100
#     noises_std = np.array([sideband_12_std[jj], sideband_18_std[jj], sideband_21_std[jj], sideband_33_std[jj], sideband_36_std[jj]])**2

    mm_tmp = ll.find_mle_multiHarmoincs(x=bb.z2, template=shaking378_template, scales=gravity_scales, 
                                          signal_freqs=gravity_freqs, bandwidth=bandwidth, noises=noises_std/noises_std[0],
                                          decimate=10, **fit_kwargs)
    print('MLEs: ', mm_tmp.values)
    A_array = np.arange(0,20,0.1)
    if mm_tmp.values[0]<0:
        PL_array = A_array*0
    else:
        PL_array = 2*10*ll.get_PL_multiHarmonics(A_array=A_array, **fit_kwargs)
        PL_array -= mm_tmp.fval*20
        for i,A_ in enumerate(A_array):
            if A_<mm_tmp.values[0]:
                PL_array[i] = 0
        
    return PL_array

In [732]:
def calc_bdf2(i):
# for i in range(0,10):
    print(i,'  ******************')
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/NoShaking_1_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    
    fit_kwargs = {'A': 0, 'phi': 0.2, 'sigma':13.85,
              'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
              'limit_phi': [0, np.pi],
              'limit_A': [-100, 100], 'limit_sigma': [0, 1000000],
              'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    
#     jj = i//100
#     noises_std = np.array([sideband_12_std[jj], sideband_18_std[jj], sideband_21_std[jj], sideband_33_std[jj], sideband_36_std[jj]])**2

    mm_tmp = ll.find_mle_multiHarmoincs(x=bb.z2, template=shaking378_template, scales=gravity_scales, 
                                          signal_freqs=gravity_freqs, bandwidth=bandwidth, noises=noises_std/noises_std[0],
                                          decimate=10, **fit_kwargs)
    print('MLEs: ', mm_tmp.values)
    A_array = np.arange(0,5,0.02)
    if mm_tmp.values[0]<0:
        PL_array = 2*10*ll.get_PL_multiHarmonics(A_array=A_array, **fit_kwargs)
        PL_array -= PL_array[0]
    else:
        PL_array = 2*10*ll.get_PL_multiHarmonics(A_array=A_array, **fit_kwargs)
        PL_array -= mm_tmp.fval*20
        for i,A_ in enumerate(A_array):
            if A_<mm_tmp.values[0]:
                PL_array[i] = 0
        
    return PL_array

In [731]:
## likelihood calculator stuff
# gravity_freqs = np.array([6])
gravity_freqs = np.array([36])
gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs, gfw.tf_freq, gfw.tf_ffts[2])
noises_std = np.ones(len(gravity_freqs))

In [730]:
tmp_PL = calc_bdf(100)
_,ax = plt.subplots()
ax.plot(np.arange(0,20,0.1), tmp_PL*2)
ax.set(ylim=(0,4), xlim=(0.0,20), xlabel=r'$\alpha \, [10^{8}]$', ylabel = 'Profile-Likelihood')

100   ******************
MLEs:  <ValueView of Minuit at 54723c38>
  A: 15.09194254917621
  phi: 0.2
  sigma: 1172.4818695464933


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(0.0, 4.0),
 (0.0, 20.0),
 Text(0.5, 0, '$\\alpha \\, [10^{8}]$'),
 Text(0, 0.5, 'Profile-Likelihood')]

In [733]:
PLs_gravity37 = np.array(Parallel(n_jobs=40)(delayed(calc_bdf2)(i) for i in tqdm(range(1000))))  # process data_inputs iterable with pool








  0%|          | 0/1000 [00:00<?, ?it/s]






  4%|▍         | 40/1000 [00:07<02:50,  5.62it/s]






  8%|▊         | 80/1000 [00:14<02:45,  5.56it/s]






 12%|█▏        | 120/1000 [00:31<03:40,  4.00it/s]






 16%|█▌        | 160/1000 [00:47<04:11,  3.34it/s]






 20%|██        | 200/1000 [01:05<04:34,  2.91it/s]






 24%|██▍       | 240/1000 [01:20<04:30,  2.81it/s]






 28%|██▊       | 280/1000 [01:37<04:27,  2.70it/s]






 32%|███▏      | 320/1000 [01:52<04:16,  2.65it/s]






 36%|███▌      | 360/1000 [02:10<04:12,  2.53it/s]






 40%|████      | 400/1000 [02:26<03:58,  2.51it/s]






 44%|████▍     | 440/1000 [02:43<03:45,  2.48it/s]






 48%|████▊     | 480/1000 [02:58<03:28,  2.50it/s]






 52%|█████▏    | 520/1000 [03:15<03:15,  2.46it/s]






 56%|█████▌    | 560/1000 [03:32<03:01,  2.42it/s]






 60%|██████    | 600/1000 [03:49<02:46,  2.40it/s]






 64%|██████▍   | 640/1000 [04:06<02:29,  2.41it/s]






 68%|██████▊   | 680/1000 [04:22<02:

In [706]:
PLs_gravity36 = np.array(Parallel(n_jobs=40)(delayed(calc_bdf)(i) for i in tqdm(range(1000))))  # process data_inputs iterable with pool








  0%|          | 0/1000 [00:00<?, ?it/s]






  4%|▍         | 40/1000 [00:22<08:53,  1.80it/s]






  8%|▊         | 80/1000 [00:22<06:00,  2.55it/s]






  8%|▊         | 80/1000 [00:37<06:00,  2.55it/s]






 12%|█▏        | 120/1000 [01:08<09:04,  1.62it/s]






 16%|█▌        | 160/1000 [01:22<07:30,  1.87it/s]






 20%|██        | 200/1000 [01:48<07:39,  1.74it/s]






 24%|██▍       | 240/1000 [02:16<07:40,  1.65it/s]






 24%|██▍       | 241/1000 [02:16<05:58,  2.12it/s]






 24%|██▍       | 242/1000 [02:16<04:56,  2.56it/s]






 24%|██▍       | 243/1000 [02:16<03:56,  3.20it/s]






 28%|██▊       | 280/1000 [03:04<07:21,  1.63it/s]






 32%|███▏      | 320/1000 [03:25<06:38,  1.71it/s]






 36%|███▌      | 360/1000 [03:36<05:14,  2.04it/s]






 40%|████      | 400/1000 [04:20<06:43,  1.49it/s]






 44%|████▍     | 440/1000 [04:51<06:34,  1.42it/s]






 48%|████▊     | 480/1000 [05:02<04:58,  1.74it/s]






 52%|█████▏    | 520/1000 [05:10<03:4

In [735]:
A_array = np.arange(0,5,0.02)
_,ax = plt.subplots()
for i in [100, 200, 500, 1000]:
    tt = np.sum(PLs_gravity37[:i], axis=0)
    ttmin = np.min(tt)
    ax.plot(A_array, tt-ttmin, label=str(i)+' datasets')
ax.legend()
ax.plot(A_array, A_array*0 + 2.71, 'r--')
ax.set(ylim=(0,10), xlim=(0.0,1), xlabel=r'$\alpha \, [10^{8}]$', ylabel = 'Profile-Likelihood')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(0.0, 10.0),
 (0.0, 1.0),
 Text(0.5, 0, '$\\alpha \\, [10^{8}]$'),
 Text(0, 0.5, 'Profile-Likelihood')]

In [694]:
_,ax = plt.subplots()
ax.scatter(range(1000), np.std(PLs_gravity34, axis=1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [696]:
data = []
for i in range(1000):
    if  np.std(PLs_gravity34[i])<0.6:
        data.append(PLs_gravity34[i])
data = np.array(data)
print(data.shape)

(999, 250)


In [708]:
A_array = np.arange(0,5,0.02)
_,ax = plt.subplots()
for i in [100,500,1000]:
    tt = np.sum(data[:i]+PLs_gravity33[:999][:i]+PLs_gravity35[:999][:i]+PLs_gravity36[:999][:i], axis=0)
    ttmin = np.min(tt)
    ax.plot(A_array, tt-ttmin, label=str(i)+' datasets')
ax.legend()
ax.plot(A_array, A_array*0 + 2.71, 'r--')
ax.set(ylim=(0,10), xlim=(0.0,5), xlabel=r'$\alpha \, [10^{8}]$', ylabel = 'Profile-Likelihood')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(0.0, 10.0),
 (0.0, 5.0),
 Text(0.5, 0, '$\\alpha \\, [10^{8}]$'),
 Text(0, 0.5, 'Profile-Likelihood')]

In [483]:
A_array = np.arange(0,5,0.02)
lim_evolution = []
for i in range(0,10000,100):
    tt = np.sum(data[:i], axis=0)
    ttmin = np.min(tt)
    lim_evolution.append(np.interp(2.71, tt-ttmin, A_array))
lim_evolution = np.array(lim_evolution)

In [484]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.semilogy(np.arange(0, 10000, 100)*10/60, lim_evolution, '--')
# ax.set(ylim=(0.2,10), xlabel='Integration time [minute]')
ax.set_ylabel(r'Limit evolution $[\alpha_8]$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Limit evolution $[\\alpha_8]$')

In [535]:
## combined
A_array = np.arange(0,5,0.02)
lim_evolution = []
for i in range(0,10000,100):
    tt = np.sum(data6[:i], axis=0)+np.sum(data12[:i], axis=0)+np.sum(data15[:i], axis=0)+np.sum(data18[:i], axis=0)
    ttmin = np.min(tt)
    lim_evolution.append(np.interp(2.71, tt-ttmin, A_array))
lim_evolution = np.array(lim_evolution)

In [537]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.semilogy(np.arange(0, 10000, 100)*10/60, lim_evolution12, '--', label='12Hz')
ax.semilogy(np.arange(0, 10000, 100)*10/60, lim_evolution15, '--', label='15Hz')
ax.semilogy(np.arange(0, 10000, 100)*10/60, lim_evolution18, '--', label='18Hz')
ax.semilogy(np.arange(0, 10000, 100)*10/60, lim_evolution6, '--', label='6Hz')
ax.semilogy(np.arange(0, 10000, 100)*10/60, lim_evolution, '--', label='all')
ax.semilogy(np.arange(0, 10000, 100)*10/60, np.arange(0, 10000, 100)*0+0.0358, 'r--', label='noise only')
ax.set(ylim=(0.2,10), xlabel='Integration time [minute]', title='multiple harmonics')
ax.set_ylabel(r'Limit evolution $[\alpha_8]$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …